## Importing Libraries

In [ ]:
# Libraries to be used for project
# Basic libraries
import pandas as pd 
import numpy as np 


#NLTK libraries
import nltk
import re
import string
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Machine Learning libraries
import sklearn 
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn import svm, datasets
from sklearn import preprocessing 


#Metrics libraries
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

#Visualization libraries
import matplotlib.pyplot as plt 
from matplotlib import rcParams
import seaborn as sns
from textblob import TextBlob
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import iplot
%matplotlib inline

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Other miscellaneous libraries
from scipy import interp
from itertools import cycle
import cufflinks as cf
from collections import defaultdict
from collections import Counter
from imblearn.over_sampling import SMOTE

## Loading of data into MongoDB

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
mydb = client["amazonPhones"] 
my_collection = mydb['phoneReviews'] 

In [5]:
import bson
import json
import dateutil 
import parser
from bson.json_util import loads

C:\Users\seanc\AppData\Local\Temp\ipykernel_14916\2693829194.py:4: DeprecationWarning: The parser module is deprecated and will be removed in future versions of Python
  import parser


In [35]:
with open('outputWithNPCountAndSentiment.json', 'r') as f:
    data = json.load(f)

all_reviews = []

for record in data:
    for review in record['reviewList']:
        all_reviews.append(review)

# test print uncomment to view
# for i in range(10):
#     print(all_reviews[i])

In [34]:
# This will throw an error (duplicate key) but it works
converted = loads(json.dumps(all_reviews))
my_collection.insert_many(converted)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 57348, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: amazonPhones.phoneReviews index: _id_ dup key: { _id: ObjectId('6416645e9a51903fe2bf93e3') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6416645e9a51903fe2bf93e3')}, 'op': {'_id': ObjectId('6416645e9a51903fe2bf93e3'), 'overall': 1, 'verified': False, 'reviewTime': '12 21, 2005', 'reviewerID': 'AJMBX04BZWPSD', 'asin': 'B0000SX3BK', 'reviewerName': 'K. Horn', 'reviewText': 'I had been very happy with my Motorola phone until the battery started losing its charge within a day, even when the phone was turned off. The friendly but incompetent customer support kept me on the phone for an average of 40 minutes for the repeated calls trying to figure out the problem. After the phone had finally been sent to their technical service center, they returned it to me that the phone was irreperable "due to customer abuse." I have done no

## Dataset Details

In [9]:
# loading into Pandas DataFrame
cursor = my_collection.find()
df = pd.DataFrame(list(cursor))

# Data set summary
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57348 entries, 0 to 57347
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               57348 non-null  object
 1   overall           57348 non-null  int64 
 2   verified          57348 non-null  bool  
 3   reviewTime        57348 non-null  object
 4   reviewerID        57348 non-null  object
 5   asin              57348 non-null  object
 6   style             26487 non-null  object
 7   reviewerName      57340 non-null  object
 8   reviewText        57316 non-null  object
 9   summary           57331 non-null  object
 10  unixReviewTime    57348 non-null  int64 
 11  nounPhrasesCount  57348 non-null  object
 12  sentiment         57316 non-null  object
 13  image             549 non-null    object
 14  vote              10806 non-null  object
dtypes: bool(1), int64(2), object(12)
memory usage: 6.2+ MB
None


In [10]:
# Sample view of the data
df.head()

,_id,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,nounPhrasesCount,sentiment,image,vote
0,641664ab9a51903fe2caffe9,5,True,"08 21, 2018",A2C8N6W0XPYJB0,B00YD547Q6,{'Color:': ' Space Gray'},Andrade2005,"very good, just what I needed, I love",good product quality,1534809600,{},"{'neg': 0.0, 'neu': 0.351, 'pos': 0.649, 'comp...",NaN,NaN
1,641664ab9a51903fe2cb0000,5,True,"07 19, 2018",A3GK98D7JMR41X,B00YD547Q6,{'Color:': ' Space Gray'},Russell V. Lewey,Bought this phone for my Mom. It looked brand...,Like New,1531958400,"{'bought': 1, 'this phone': 1, 'my mom': 1, 'b...","{'neg': 0.055, 'neu': 0.775, 'pos': 0.17, 'com...",NaN,NaN
2,641664ab9a51903fe2cb0009,1,True,"07 3, 2018",AQHXZAFT2UTS8,B00YD547Q6,{'Color:': ' Space Gray'},Zero,The phone which should be cmda unlocked would ...,Stay away from these on Verizon,1530576000,"{'the phone': 1, 'verizon worked fine': 1, 'a ...","{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp...",NaN,NaN
3,641664ab9a51903fe2cb000c,5,True,"07 2, 2018",A3OFF4F1AZHOPE,B00YD547Q6,{'Color:': ' Gold'},Delroy Jones,Good,Nice,1530489600,{},"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",NaN,NaN
4,641664ab9a51903fe2cb0010,5,True,"06 23, 2018",A1NNNEPCM14OGF,B00YD547Q6,{'Color:': ' Silver'},Erick R!,Very good!,Five Stars,1529712000,{},"{'neg': 0.0, 'neu': 0.223, 'pos': 0.777, 'comp...",NaN,NaN


## Preprocessing and cleaning

In [17]:
process_reviews = df.copy()
process_reviews.isnull().sum()

_id                     0
overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style               30861
reviewerName            8
reviewText             32
summary                17
unixReviewTime          0
nounPhrasesCount        0
sentiment              32
image               56799
vote                46542
dtype: int64

In [18]:
# Handle null values
process_reviews['reviewText']=process_reviews['reviewText'].fillna('Missing')

# Concatenating review text and summary
process_reviews['reviews']=process_reviews['reviewText']+process_reviews['summary']

## Remove all irrelevant fields
process_reviews=process_reviews.drop(['image', 'vote', 'summary'], axis=1)

# 
process_reviews.head()

,_id,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,unixReviewTime,nounPhrasesCount,sentiment,reviews
0,641664ab9a51903fe2caffe9,5,True,"08 21, 2018",A2C8N6W0XPYJB0,B00YD547Q6,{'Color:': ' Space Gray'},Andrade2005,"very good, just what I needed, I love",1534809600,{},"{'neg': 0.0, 'neu': 0.351, 'pos': 0.649, 'comp...","very good, just what I needed, I lovegood prod..."
1,641664ab9a51903fe2cb0000,5,True,"07 19, 2018",A3GK98D7JMR41X,B00YD547Q6,{'Color:': ' Space Gray'},Russell V. Lewey,Bought this phone for my Mom. It looked brand...,1531958400,"{'bought': 1, 'this phone': 1, 'my mom': 1, 'b...","{'neg': 0.055, 'neu': 0.775, 'pos': 0.17, 'com...",Bought this phone for my Mom. It looked brand...
2,641664ab9a51903fe2cb0009,1,True,"07 3, 2018",AQHXZAFT2UTS8,B00YD547Q6,{'Color:': ' Space Gray'},Zero,The phone which should be cmda unlocked would ...,1530576000,"{'the phone': 1, 'verizon worked fine': 1, 'a ...","{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'comp...",The phone which should be cmda unlocked would ...
3,641664ab9a51903fe2cb000c,5,True,"07 2, 2018",A3OFF4F1AZHOPE,B00YD547Q6,{'Color:': ' Gold'},Delroy Jones,Good,1530489600,{},"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",GoodNice
4,641664ab9a51903fe2cb0010,5,True,"06 23, 2018",A1NNNEPCM14OGF,B00YD547Q6,{'Color:': ' Silver'},Erick R!,Very good!,1529712000,{},"{'neg': 0.0, 'neu': 0.223, 'pos': 0.777, 'comp...",Very good!Five Stars
